<h1 align ="center"> REST API Mutiple Image Samples</h1>
<hr>
   
# Chat Completions

In [1]:
import json
import os
import requests
import base64

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup deployment_name, openai_api_base, openai_api_key and openai_api_version.

In [2]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)
    
# Setting up the deployment name
deployment_name = config_details['GPT-4V_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Currently OPENAI API have the following versions available: 2022-12-01. All versions follow the YYYY-MM-DD date structure.
openai_api_version = config_details['OPENAI_API_VERSION']

In [3]:

#Defect Detector
reference_image_file_path = "../../common/images/DefectDetectorReference.jfif" 
with open(reference_image_file_path, 'rb') as image_file:
    encoded_reference_image = base64.b64encode(image_file.read()).decode('utf-8')

test_image_file_path = "../../common/images/DefectDetectorTest.jfif"
with open(test_image_file_path, 'rb') as image_file:
    encoded_test_image = base64.b64encode(image_file.read()).decode('utf-8')

sys_message = """
You're a professional defect detector.
Your job is to compare the test image with reference image, please answer the question with "No defect detected" or "Defect detected", 
also explain your decision as detail as possible."
"""

# Construct the API request URL
api_url = f"{openai_api_base}/openai/deployments/{deployment_name}/chat/completions?api-version={openai_api_version}"

# Including the api-key in HTTP headers
headers = {
    "Content-Type": "application/json",
    "api-key": openai_api_key,
}

# Payload for the request
payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "system",
      "content": [
          {
            "type": "text", 
            "text": sys_message
          }
      ]
    },
    {
      "role": "user",
      "content": [
          {
            "type": "text",
            "text": "Here is the reference image", # Pass the prompt
          },
          {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_reference_image}" #Pass the encoded reference image
            }
          },
          {
            "type": "text",
            "text": "Here is the test image", # Pass the prompt
          },
          {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_test_image}" #Pass the encoded test image
            }
          }
      ]
    }    
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

# Send the request and handle the response
try:
    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad HTTP status codes
    response_content = response.json()
    print(response_content['choices'][0]['message']['content'])  # Print the content of the response
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

Defect detected.

The test image shows a can lid with visible deformations. There are areas where the material appears to be peeled or chipped off, especially around the center near the pull tab. This is a clear deviation from the reference image, which shows a flawless, smooth surface with no imperfections. The reference image displays a pristine can lid with an intact surface and an unblemished appearance, whereas the test image exhibits damage that would be considered a defect in a quality control assessment.
